# Flowers102: Ensemble Experiments (K-fold + Averaged Softmax)

Build and evaluate ensemble models on the Flowers102 dataset, covering three configurations: baseline, greedy tuning, and Optuna tuning. Provide a reproducible training and testing workflow.

- **K-fold Deterministic Training and Early Stopping**: Train sub-models fold by fold, fixing randomness, stopping early based on validation loss, and saving the optimal weights for each fold.

- **Batch Running of Ensemble Experiments**: Unified triggering of multi-fold training and logging.

- **Testing and Evaluating the Ensemble**: Calculate the accuracy and macro/weighted metrics by arithmetically averaging the softmax probabilities of each fold's output, and write the results to a JSON summary.

- **12-fold Greedy Tuning Version Training and Comparison**: Train a 12-fold ensemble using selected hyperparameters and compare it with the baseline 12-fold ensemble.

- **Large Training Set Experiment with Merged Train + Values**: Construct a unified DataLoader, train and evaluate on the merged data, and update the JSON summary.

- **12-fold Optuna Hyperparameter Tuning Version**: This version trains and evaluates the Optuna architecture using 12 folds and a large training set, comparing it with other configurations 

- **Siamese 5 shot Optuna Hyperparameter Version**: This tries to train 12 Siamese models using the Optuna hyperparameters and same ensemble training as other 12-fold ensemble models and compares it against 12-fold Optuna Hyperparameter Version 

In [1]:
import os, json, random, numpy as np, torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from tqdm import tqdm


# Baseline Flowers102 pipeline setup
 - Initialize the baseline Flowers102 classification experiment using shared utilities from `flowers_common`.
 - Import helpers for seeding, device selection, data loading, model construction, training, evaluation, and inference.
 - Fix random seeds to ensure deterministic behavior across Python, NumPy, and PyTorch.
 - Select the compute device (CPU, CUDA, or MPS) via `get_device_config`.
 - Build deterministic train/validation/test `DataLoader` objects for the Flowers102 dataset.

In [2]:
from flowers_common import seed_all, get_device_config, get_dataloaders
from flowers_common import build_resnet50_cosine

seed_all(1029)
dc = get_device_config()
build_model = build_resnet50_cosine
device = dc.device
train_loader, val_loader, test_loader = get_dataloaders(root="data", batch_size=32, img_size=224)

c:\Users\Eugene Ian\OneDrive\Desktop\NTU Y4 S1 notes\SC4001 Neural Network and Deep Learning\Group Assignment\Notebook\.venv\Lib\site-packages\torch\backends\__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:85.)
  self.setter(val)


## 3.2.1: Ensemble Experiment with Training Loop with Early Stopping. 

**Objective**: To improve generalization by performing K-fold training with fixed randomness to obtain multiple independent sub-models. Early stopping is used to avoid overfitting and ineffective training.

**Method**:

- Divide the training set into K folds. Select k-1 folds for training and then validate against validation set

- Use the same random seed, data augmentation, and optimization strategy for each fold.

- Monitor the validation set loss and save the optimal weights for that fold when the early stopping condition is met.

- Output the metrics and checkpoint paths for each fold for subsequent ensemble inference. 

In [ ]:
# =============================================================================
# 3.2.1. Deterministic K-Fold Ensemble Training with Early Stopping
# =============================================================================

# ---- Device requirements ----
assert torch.cuda.is_available(), "CUDA is not available."
device = torch.device("cuda:0")
torch.backends.cudnn.benchmark = False  # deterministic
torch.backends.cudnn.deterministic = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# ---- Global determinism ----
SEED = 1029
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True, warn_only=False)


# ---- Base trainer (deterministic setup) ----
def train_model(model, train_loader, val_loader, epochs=20, lr=1e-4, patience=5, ckpt_path="ensemble_ckpt/best.pth"):
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scaler = GradScaler(enabled=True)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        g = torch.Generator().manual_seed(SEED + epoch)  # Deterministic shuffle per epoch
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=True):
                logits = model(imgs)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        # ---- Validation ----
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                preds = logits.argmax(1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | Train {train_loss:.4f}/{train_acc:.4f} | Val {val_loss:.4f}/{val_acc:.4f}")

        # ---- Early Stopping ----
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), ckpt_path)
            print(f" Saved best model → {ckpt_path}")
        else:
            patience_counter += 1
            print(f" EarlyStopping counter {patience_counter}/{patience}")
            if patience_counter >= patience:
                print(" Early stopping triggered.")
                break


# ---- K-fold experiment runner ----
def run_kfold_training(
    k: int,
    model_fn,
    full_dataset,
    epochs=20,
    lr=1e-4,
    patience=5,
    base_ckpt_dir="ensemble_ckpt",
    metrics_path="ensemble_ckpt/metrics.json",
    random_seed=SEED
):
    os.makedirs(base_ckpt_dir, exist_ok=True)
    k_dir = os.path.join(base_ckpt_dir, f"k{k}")
    os.makedirs(k_dir, exist_ok=True)

    # Load or initialize persistent metrics JSON
    if os.path.exists(metrics_path):
        with open(metrics_path, "r") as f:
            metrics = json.load(f)
    else:
        metrics = {}
    metrics.setdefault(f"k{k}", [])

    # Split indices deterministically
    if k == 1:
        fold_indices = [(list(range(len(full_dataset))), list(range(len(full_dataset))))]
    else:
        kf = KFold(n_splits=k, shuffle=True, random_state=random_seed)
        fold_indices = list(kf.split(range(len(full_dataset))))

    print(f"\n=== Running {k}-Fold Ensemble Training ===")

    for fold, (train_idx, val_idx) in enumerate(fold_indices, 1):
        print(f"\n[Fold {fold}/{k}] Train={len(train_idx)} | Val={len(val_idx)}")

        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        g = torch.Generator().manual_seed(random_seed + fold)
        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True,
                                  num_workers=0, pin_memory=True, generator=g)
        fold_val_loader = DataLoader(val_subset, batch_size=64, shuffle=False,
                                     num_workers=0, pin_memory=True, generator=g)

        model = model_fn()
        fold_ckpt_path = os.path.join(k_dir, f"model_fold{fold}.pth")

        # Train model (with early stopping)
        train_model(model, train_loader, fold_val_loader, epochs, lr, patience, fold_ckpt_path)

        # Evaluate on validation fold
        model.load_state_dict(torch.load(fold_ckpt_path, map_location=device))
        model.eval()
        total_loss, correct, total = 0.0, 0, 0
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in fold_val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                total_loss += loss.item()
                preds = logits.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_loss = total_loss / len(fold_val_loader)
        val_acc = correct / total
        print(f"[Fold {fold}] Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

        metrics[f"k{k}"].append({
            "fold": fold,
            "val_loss": round(val_loss, 4),
            "val_acc": round(val_acc, 4),
            "ckpt_path": fold_ckpt_path
        })

        with open(metrics_path, "w") as f:
            json.dump(metrics, f, indent=2)

    print(f"\n=== k={k} training complete. Metrics saved → {metrics_path} ===")
    return metrics

# === Section 4C: Run ensemble experiments ===
for k in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]: ##  <- modify this to to train different k fold setups.
    run_kfold_training(
        k=k,
        model_fn=lambda: build_resnet50_cosine(num_classes=102, pretrained=True),
        full_dataset=train_loader.dataset,
        epochs=30,
        lr=1e-4,
        patience=5,
        base_ckpt_dir="ensemble_ckpt",
        metrics_path="ensemble_ckpt/metrics.json"
    )



=== Running 1-Fold Ensemble Training ===

[Fold 1/1] Train=1020 | Val=1020


C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\424728035.py:34: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=True)
Epoch 1/30:   0%|          | 0/32 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\424728035.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\424728035.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=True):


Epoch 01 | Train 4.5536/0.0422 | Val 3.8972/0.2029
 Saved best model → ensemble_ckpt\k1\model_fold1.pth


Epoch 02 | Train 3.5493/0.3451 | Val 2.8662/0.6245
 Saved best model → ensemble_ckpt\k1\model_fold1.pth


Epoch 03 | Train 2.6027/0.6745 | Val 1.9414/0.8373
 Saved best model → ensemble_ckpt\k1\model_fold1.pth


...

Epoch 20 | Train 0.0661/0.9985 | Val 0.9201/0.8206
 EarlyStopping counter 1/5


Epoch 21 | Train 0.0467/1.0000 | Val 0.8976/0.8059
 EarlyStopping counter 2/5


Epoch 22 | Train 0.0488/0.9985 | Val 0.9019/0.7971
 EarlyStopping counter 3/5


Epoch 23 | Train 0.0457/1.0000 | Val 0.8901/0.8000
 EarlyStopping counter 4/5


Epoch 24 | Train 0.0389/1.0000 | Val 0.9180/0.7853
 EarlyStopping counter 5/5
 Early stopping triggered.
[Fold 3] Val Loss=0.8917, Val Acc=0.8294

=== k=3 training complete. Metrics saved → ensemble_ckpt/metrics.json ===


## 3.2.2: Ensemble Test Evaluations


**Objective**: To evaluate the performance of an ensemble model on a test set and generate a standardized summary of results.

**Method**:

- Load the best model for each fold and calculate the softmax probability on the test set.

- Calculate the arithmetic mean of the probabilities of each model for the same sample to obtain the ensemble probability.

- Calculate the accuracy and macro/weighted metrics based on the ensemble probability and append the results to a JSON summary.

In [4]:
# =============================================================================
# 3.2.2. Ensemble Test Evaluation (averaged softmax + JSON summary)
# =============================================================================
import os, re, json, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

BASE_DIR = "ensemble_ckpt"
EVAL_PATH = os.path.join(BASE_DIR, "eval_results.json")
NUM_CLASSES = 102

# ------------------------------------------------------------------
def _find_k_dirs(base_dir=BASE_DIR):
    ks = []
    if not os.path.isdir(base_dir):
        return ks
    for name in os.listdir(base_dir):
        m = re.fullmatch(r"k(\d+)", name)
        if m:
            k = int(m.group(1))
            k_dir = os.path.join(base_dir, name)
            has_model = any(fn.endswith(".pth") for fn in os.listdir(k_dir))
            if has_model:
                ks.append(k)
    return sorted(ks)

def _load_models_for_k(k, base_dir=BASE_DIR, num_classes=NUM_CLASSES):
    k_dir = os.path.join(base_dir, f"k{k}")
    ckpts = sorted([os.path.join(k_dir, fn) for fn in os.listdir(k_dir) if fn.endswith(".pth")])
    models = []
    for path in ckpts:
        m = build_resnet50_cosine(num_classes=num_classes, pretrained=True)
        state = torch.load(path, map_location=device)
        m.load_state_dict(state)
        m.to(device).eval()
        models.append(m)
    return models, ckpts

def _ensemble_predict(models, imgs):
    """Average softmax probabilities across models."""
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
        probs_sum = None
        for m in models:
            logits = m(imgs)
            probs = torch.softmax(logits, dim=1)
            probs_sum = probs if probs_sum is None else (probs_sum + probs)
        probs_mean = probs_sum / len(models)
    return probs_mean.detach().cpu()

# ------------------------------------------------------------------
# Load or init summary JSON
if os.path.exists(EVAL_PATH):
    with open(EVAL_PATH, "r") as f:
        eval_results = json.load(f)
else:
    eval_results = {}

ks_available = _find_k_dirs(BASE_DIR)
if not ks_available:
    print(f"[WARN] No k-fold checkpoints found under '{BASE_DIR}'.")
else:
    print(f"[INFO] Found k-folds to evaluate: {ks_available}")

for k in ks_available:
    print(f"\n=== Evaluating k={k} ensemble ===")
    models, ckpts = _load_models_for_k(k)
    print(f"[INFO] Loaded {len(models)} model(s):")
    for p in ckpts:
        print(f"  - {p}")

    all_preds, all_labels = [], []

    for imgs, labels in tqdm(test_loader, desc=f"Testing (k={k})"):
        imgs = imgs.to(device, non_blocking=True)
        probs_mean = _ensemble_predict(models, imgs)
        preds = probs_mean.argmax(axis=1).numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

    test_acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1 = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"\n[k={k}] Test Accuracy: {test_acc:.4f} | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f}")

    # Record summary to JSON (no classwise)
    eval_results[f"k{k}"] = {
        "num_models": len(models),
        "test_accuracy": round(test_acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
    }

    with open(EVAL_PATH, "w") as f:
        json.dump(eval_results, f, indent=2)

    # Free GPU for next k
    for m in models:
        del m
    torch.cuda.empty_cache()

print(f"\n Evaluation complete. Results saved to {EVAL_PATH}")

[INFO] Found k-folds to evaluate: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

=== Evaluating k=1 ensemble ===
[INFO] Loaded 1 model(s):
  - ensemble_ckpt\k1\model_fold1.pth


Testing (k=1):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1469685463.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
Testing (k=1): 100%|██████████| 97/97 [00:38<00:00,  2.50it/s]



[k=1] Test Accuracy: 0.9003 | Macro F1: 0.8947 | Weighted F1: 0.9006

=== Evaluating k=2 ensemble ===
[INFO] Loaded 2 model(s):
  - ensemble_ckpt\k2\model_fold1.pth
  - ensemble_ckpt\k2\model_fold2.pth


Testing (k=2):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1469685463.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
Testing (k=2): 100%|██████████| 97/97 [00:33<00:00,  2.90it/s]



[k=2] Test Accuracy: 0.8743 | Macro F1: 0.8695 | Weighted F1: 0.8755

=== Evaluating k=3 ensemble ===
[INFO] Loaded 3 model(s):
  - ensemble_ckpt\k3\model_fold1.pth
  - ensemble_ckpt\k3\model_fold2.pth
  - ensemble_ckpt\k3\model_fold3.pth


...


[k=13] Test Accuracy: 0.9260 | Macro F1: 0.9258 | Weighted F1: 0.9261

=== Evaluating k=14 ensemble ===
[INFO] Loaded 14 model(s):
  - ensemble_ckpt\k14\model_fold1.pth
  - ensemble_ckpt\k14\model_fold10.pth
  - ensemble_ckpt\k14\model_fold11.pth
  - ensemble_ckpt\k14\model_fold12.pth
  - ensemble_ckpt\k14\model_fold13.pth
  - ensemble_ckpt\k14\model_fold14.pth
  - ensemble_ckpt\k14\model_fold2.pth
  - ensemble_ckpt\k14\model_fold3.pth
  - ensemble_ckpt\k14\model_fold4.pth
  - ensemble_ckpt\k14\model_fold5.pth
  - ensemble_ckpt\k14\model_fold6.pth
  - ensemble_ckpt\k14\model_fold7.pth
  - ensemble_ckpt\k14\model_fold8.pth
  - ensemble_ckpt\k14\model_fold9.pth


Testing (k=14):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1469685463.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
Testing (k=14): 100%|██████████| 97/97 [01:08<00:00,  1.43it/s]



[k=14] Test Accuracy: 0.9284 | Macro F1: 0.9276 | Weighted F1: 0.9287

=== Evaluating k=15 ensemble ===
[INFO] Loaded 15 model(s):
  - ensemble_ckpt\k15\model_fold1.pth
  - ensemble_ckpt\k15\model_fold10.pth
  - ensemble_ckpt\k15\model_fold11.pth
  - ensemble_ckpt\k15\model_fold12.pth
  - ensemble_ckpt\k15\model_fold13.pth
  - ensemble_ckpt\k15\model_fold14.pth
  - ensemble_ckpt\k15\model_fold15.pth
  - ensemble_ckpt\k15\model_fold2.pth
  - ensemble_ckpt\k15\model_fold3.pth
  - ensemble_ckpt\k15\model_fold4.pth
  - ensemble_ckpt\k15\model_fold5.pth
  - ensemble_ckpt\k15\model_fold6.pth
  - ensemble_ckpt\k15\model_fold7.pth
  - ensemble_ckpt\k15\model_fold8.pth
  - ensemble_ckpt\k15\model_fold9.pth


Testing (k=15):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1469685463.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
Testing (k=15): 100%|██████████| 97/97 [01:11<00:00,  1.35it/s]


[k=15] Test Accuracy: 0.9291 | Macro F1: 0.9290 | Weighted F1: 0.9293

 Evaluation complete. Results saved to ensemble_ckpt\eval_results.json


## 3.2.3: 12-Fold Model Training with greedy optimal hyperparameters



**Objective**: To build a stronger ensemble foundation by performing 12-fold training using tuned hyperparameters.

**Methods**:

- Load the optimal set of hyperparameters from the tuning phase.

- Repeat the "deterministic training + early stopping" process on the 12-fold partition.

- Save the best checkpoint and validation metrics for each fold.

In [5]:
# =============================================================================
# 3.2.3. 12-Fold training with tuned hyperparameters (deterministic, early stopping)
# =============================================================================
import os, json, random, numpy as np, torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from tqdm import tqdm

# ---------------------------
# Tuned hyperparameters
# ---------------------------
HP = {
    "lr": 5e-4,
    "weight_decay": 1e-4,
    "s": 16.0,
    "label_smoothing": 0.15,
    "batch_size": 16,
    "val_batch_size": 32,
    "epochs": 30,
    "patience": 5,
}

BASE_CKPT_DIR = "12fold_ensemble_ckpt"
METRICS_PATH  = os.path.join(BASE_CKPT_DIR, "metrics.json")

# ---------------------------
# Determinism (same as before)
# ---------------------------
assert torch.cuda.is_available(), "CUDA is not available."
device = torch.device("cuda:0")

SEED = 1029
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True, warn_only=False)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# ---------------------------
# Trainer (adds wd + label smoothing)
# ---------------------------
def train_model_hp(model, train_loader, val_loader, *, epochs, lr, weight_decay, label_smoothing, patience, ckpt_path):
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scaler = GradScaler(enabled=True)

    best_val_loss, patience_counter = float("inf"), 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=True):
                logits = model(imgs)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc  = correct / total

        # ----- Validation -----
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | Train {train_loss:.4f}/{train_acc:.4f} | Val {val_loss:.4f}/{val_acc:.4f}")

        # Early stopping + save
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), ckpt_path)
            print(f"  Saved best → {ckpt_path}")
        else:
            patience_counter += 1
            print(f"  EarlyStopping counter {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("  Early stopping triggered.")
                break

# ---------------------------
# Single run for k=12
# ---------------------------
def run_k12_with_tuned_hp(full_dataset, model_build_fn):
    os.makedirs(BASE_CKPT_DIR, exist_ok=True)
    # load/initialize metrics file
    metrics = {}
    if os.path.exists(METRICS_PATH):
        with open(METRICS_PATH, "r") as f:
            try:
                metrics = json.load(f)
            except Exception:
                metrics = {}
    metrics["k12_tuned"] = []

    # Split deterministically
    kf = KFold(n_splits=12, shuffle=True, random_state=SEED)
    fold_indices = list(kf.split(range(len(full_dataset))))
    k_dir = os.path.join(BASE_CKPT_DIR, "k12_greedy_HP")
    os.makedirs(k_dir, exist_ok=True)

    print("\n=== Running 12-Fold training with tuned hyperparameters ===")
    for fold, (tr_idx, va_idx) in enumerate(fold_indices, 1):
        print(f"\n[Fold {fold}/12] Train={len(tr_idx)} | Val={len(va_idx)}")

        train_subset = Subset(full_dataset, tr_idx)
        val_subset   = Subset(full_dataset, va_idx)

        g = torch.Generator().manual_seed(SEED + fold)
        train_loader = DataLoader(train_subset, batch_size=HP["batch_size"], shuffle=True,
                                  num_workers=0, pin_memory=True, generator=g)
        val_loader   = DataLoader(val_subset,   batch_size=HP["val_batch_size"], shuffle=False,
                                  num_workers=0, pin_memory=True, generator=g)

        # Build model and set cosine scale s=16
        model = model_build_fn()
        # model[1] is the CosineClassifier in your build_model
        if hasattr(model[1], "s"):
            model[1].s = HP["s"]

        fold_ckpt_path = os.path.join(k_dir, f"model_fold{fold}.pth")

        # Train with tuned HPs
        train_model_hp(
            model, train_loader, val_loader,
            epochs=HP["epochs"],
            lr=HP["lr"],
            weight_decay=HP["weight_decay"],
            label_smoothing=HP["label_smoothing"],
            patience=HP["patience"],
            ckpt_path=fold_ckpt_path
        )

        # Quick post-eval on the fold val set (for logging)
        model.load_state_dict(torch.load(fold_ckpt_path, map_location=device))
        model.eval()
        criterion = nn.CrossEntropyLoss(label_smoothing=HP["label_smoothing"])
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                total_loss += loss.item()
                correct += (logits.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_loss = total_loss / len(val_loader)
        val_acc  = correct / total
        print(f"  [Fold {fold}] Val Loss={val_loss:.4f} | Val Acc={val_acc:.4f}")

        metrics["k12_tuned"].append({
            "fold": fold,
            "val_loss": round(val_loss, 4),
            "val_acc": round(val_acc, 4),
            "ckpt_path": fold_ckpt_path,
            "hp": {k: (float(v) if isinstance(v, (int, float)) else v) for k, v in HP.items()}
        })

        with open(METRICS_PATH, "w") as f:
            json.dump(metrics, f, indent=2)

    print(f"\n=== 12-fold tuned training complete. Metrics → {METRICS_PATH} ===")
    return metrics

# ---- Run it (build_model must be defined above) ----
run_k12_with_tuned_hp(
    full_dataset=train_loader.dataset,
    model_build_fn=lambda: build_resnet50_cosine(num_classes=102, pretrained=True)
)



=== Running 12-Fold training with tuned hyperparameters ===

[Fold 1/12] Train=935 | Val=85


C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3514006930.py:59: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=True)
Epoch 1/30:   0%|          | 0/59 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3514006930.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3514006930.py:88: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=True):


Epoch 01 | Train 4.0807/0.2257 | Val 3.4021/0.4471
  Saved best → 12fold_ensemble_ckpt\k12_greedy_HP\model_fold1.pth


Epoch 02 | Train 2.5608/0.6717 | Val 2.5291/0.6235
  Saved best → 12fold_ensemble_ckpt\k12_greedy_HP\model_fold1.pth


Epoch 03 | Train 1.8464/0.8310 | Val 2.0593/0.7059
  Saved best → 12fold_ensemble_ckpt\k12_greedy_HP\model_fold1.pth


...

Epoch 23 | Train 1.1412/0.9957 | Val 1.4261/0.8941
  EarlyStopping counter 2/5


Epoch 24 | Train 1.1225/1.0000 | Val 1.4408/0.9176
  EarlyStopping counter 3/5


Epoch 25 | Train 1.1217/0.9989 | Val 1.4068/0.9176
  EarlyStopping counter 4/5


Epoch 26 | Train 1.1224/1.0000 | Val 1.4376/0.9176
  EarlyStopping counter 5/5
  Early stopping triggered.
  [Fold 12] Val Loss=1.4298 | Val Acc=0.9294

=== 12-fold tuned training complete. Metrics → 12fold_ensemble_ckpt\metrics.json ===


{'k12_tuned': [{'fold': 1,
   'val_loss': 1.4946,
   'val_acc': 0.8824,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_greedy_HP\\model_fold1.pth',
   'hp': {'lr': 0.0005,
    'weight_decay': 0.0001,
    's': 16.0,
    'label_smoothing': 0.15,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
    'patience': 5.0}},
  {'fold': 2,
   'val_loss': 1.5303,
   'val_acc': 0.8941,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_greedy_HP\\model_fold2.pth',
   'hp': {'lr': 0.0005,
    'weight_decay': 0.0001,
    's': 16.0,
    'label_smoothing': 0.15,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
    'patience': 5.0}},
  {'fold': 3,
   'val_loss': 1.4591,
   'val_acc': 0.8824,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_greedy_HP\\model_fold3.pth',
   'hp': {'lr': 0.0005,
    'weight_decay': 0.0001,
    's': 16.0,
    'label_smoothing': 0.15,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
    'patience': 5.0}},
  {'fold': 4,
   'val

## 3.2.4: Baseline 12-fold model vs Greedy Optimal 12-fold model

**Objective:** To compare the ensemble performance differences between a 12-fold baseline configuration and a greedy hyperparameter tuning configuration on the test set.

**Methods:**

- Perform ensemble inference (average softmax) on both 12-fold models separately.

- Calculate and compare accuracy and macro/weighted metrics.

- Output the comparison results to a unified JSON summary.

In [6]:
# =============================================================================
# 3.2.4. Compare k12 baseline vs k12_greedy_HP (averaged softmax)
# =============================================================================
import os, json, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

assert 'build_model' in globals(), "build_model(...) must be defined above."
assert 'test_loader' in globals(), "test_loader must exist."
assert 'device' in globals(), "device must be set."

DIR_BASELINE = "ensemble_ckpt/k12"
DIR_TUNED    = "12fold_ensemble_ckpt/k12_greedy_HP"
OUT_JSON     = os.path.join("12fold_ensemble_ckpt", "12fold_eval_results.json")
NUM_CLASSES  = 102

def load_models_from_dir(model_dir, num_classes=NUM_CLASSES):
    ckpts = sorted([os.path.join(model_dir, f) for f in os.listdir(model_dir) if f.endswith(".pth")])
    if not ckpts:
        raise FileNotFoundError(f"No .pth files in {model_dir}")
    models = []
    for p in ckpts:
        m = build_resnet50_cosine(num_classes=num_classes, pretrained=True)
        state = torch.load(p, map_location=device)
        m.load_state_dict(state)
        m.to(device).eval()
        models.append(m)
    return models, ckpts

@torch.no_grad()
def ensemble_probs(models, imgs):
    with torch.cuda.amp.autocast(enabled=True):
        acc = None
        for m in models:
            logits = m(imgs)
            probs  = torch.softmax(logits, dim=1)
            acc = probs if acc is None else (acc + probs)
        return (acc / len(models)).detach().cpu().numpy()

def evaluate_dir(model_dir, label):
    models, ckpts = load_models_from_dir(model_dir)
    print(f"\n=== Evaluating {label} ===")
    print(f"Loaded {len(models)} models from: {model_dir}")

    all_preds, all_labels = [], []
    for imgs, labels in tqdm(test_loader, desc=f"Testing ({label})"):
        imgs = imgs.to(device, non_blocking=True)
        probs = ensemble_probs(models, imgs)
        preds = probs.argmax(axis=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.numpy().tolist())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1    = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"[{label}] Test Acc: {acc:.4f} | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f}")

    # free GPU
    for m in models:
        del m
    torch.cuda.empty_cache()

    return {
        "num_models": len(ckpts),
        "test_accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
        "models": ckpts,  # paths for traceability
    }

results = {}
results["k12_baseline"]    = evaluate_dir(DIR_BASELINE, "k12_baseline")
results["k12_greedy_HP"]   = evaluate_dir(DIR_TUNED,    "k12_greedy_HP")

os.makedirs(os.path.dirname(OUT_JSON), exist_ok=True)
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\n Saved comparison results to {OUT_JSON}")



=== Evaluating k12_baseline ===
Loaded 12 models from: ensemble_ckpt/k12


Testing (k12_baseline):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\4123415693.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing (k12_baseline): 100%|██████████| 97/97 [01:04<00:00,  1.49it/s]


[k12_baseline] Test Acc: 0.9315 | Macro F1: 0.9304 | Weighted F1: 0.9318

=== Evaluating k12_greedy_HP ===
Loaded 12 models from: 12fold_ensemble_ckpt/k12_greedy_HP


Testing (k12_greedy_HP):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\4123415693.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing (k12_greedy_HP): 100%|██████████| 97/97 [01:01<00:00,  1.58it/s]

[k12_greedy_HP] Test Acc: 0.9346 | Macro F1: 0.9328 | Weighted F1: 0.9350

 Saved comparison results to 12fold_ensemble_ckpt\12fold_eval_results.json


## 3.2.5: Combine train and validation datasets into one unified loader 

**Objective:** To merge the training and validation sets to create a larger training set, thereby increasing the capacity of the final model.

**Method:**

- Merge the original train/val samples and labels, and reconstruct the Dataset and DataLoader.

- Inherit the original normalization and data augmentation settings.

In [7]:
# =============================================================================
# 3.2.5: Combine train and validation datasets into one unified loader 
# =============================================================================
from torch.utils.data import ConcatDataset, DataLoader

# Ensure train_loader and val_loader exist
assert "train_loader" in globals() and "val_loader" in globals(), "train_loader and val_loader must be defined first."

# Combine underlying datasets
combined_dataset = ConcatDataset([train_loader.dataset, val_loader.dataset])

# Create a new DataLoader
combined_train_loader = DataLoader(
    combined_dataset,
    batch_size=32,        # can change to tuned HP batch size if desired (e.g., 16)
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

print("Combined dataset created")
print(f"Total samples: {len(combined_dataset)}")
print(f"Batch size: {combined_train_loader.batch_size}")



Combined dataset created
Total samples: 2040
Batch size: 32


## 3.2.6: Train 12-fold large-train models (baseline vs greedy HP)

**Objective:** To train two sets of 12-fold models, one for baseline and one for the greedy tuning, on the merged large training set. 

**Method:**
- Repeat the 12-fold process on the merged data for each hyperparameter configuration (or follow a fixed number of rounds/early stopping strategy set in the code).

- Because validation set is now part of the training data, use 1 hold out fold for validation for each of the 12 model. 

- Save the optimal weights and training logs by fold.

In [8]:
# =============================================================================
# 3.2.6. Train 12-fold large-train models (baseline vs tuned HP)
# =============================================================================
import os, json, torch, numpy as np
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from tqdm import tqdm

# Requirements from earlier sections
assert "combined_dataset" in globals(), "combined_dataset must be defined from Section 6C"
assert "build_model" in globals(), "build_model must be defined above"
assert "device" in globals(), "device must be set above"

SEED = 1029
BASE_DIR = "12fold_ensemble_ckpt"
OUT_BASELINE = os.path.join(BASE_DIR, "k12_baseline_largetrain")
OUT_GREEDY   = os.path.join(BASE_DIR, "k12_greedy_HP_largetrain")
os.makedirs(OUT_BASELINE, exist_ok=True)
os.makedirs(OUT_GREEDY, exist_ok=True)

# Only these five hyperparameters differ
BASELINE_HP = {
    "lr": 1e-4,
    "weight_decay": 0.0,
    "s": 30.0,
    "label_smoothing": 0.0,
    "batch_size": 32
}
GREEDY_HP = {
    "lr": 5e-4,
    "weight_decay": 1e-4,
    "s": 16.0,
    "label_smoothing": 0.15,
    "batch_size": 16
}

def train_one_fold(model, train_loader, val_loader, *, lr, weight_decay, label_smoothing, epochs=30, patience=5, ckpt_path):
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2)
    scaler = GradScaler(enabled=True)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=True):
                logits = model(imgs)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | Train {train_loss:.4f}/{train_acc:.4f} | Val {val_loss:.4f}/{val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), ckpt_path)
            print(f"Saved best to {ckpt_path}")
        else:
            patience_counter += 1
            print(f"EarlyStopping counter {patience_counter}/5")
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

def run_k12_largetrain(out_dir, hp, *, epochs=30, patience=5):
    metrics_path = os.path.join(out_dir, "metrics.json")
    metrics = []

    kf = KFold(n_splits=12, shuffle=True, random_state=SEED)
    indices = np.arange(len(combined_dataset))

    print(f"Starting 12-fold large-train into {out_dir}")
    for fold, (train_idx, val_idx) in enumerate(kf.split(indices), 1):
        print(f"Fold {fold}/12 Train {len(train_idx)} Val {len(val_idx)}")

        train_subset = Subset(combined_dataset, train_idx.tolist())
        val_subset   = Subset(combined_dataset, val_idx.tolist())

        g = torch.Generator().manual_seed(SEED + fold)
        train_loader = DataLoader(
            train_subset, batch_size=hp["batch_size"], shuffle=True,
            num_workers=0, pin_memory=True, generator=g
        )
        val_loader = DataLoader(
            val_subset, batch_size=64, shuffle=False,
            num_workers=0, pin_memory=True, generator=g
        )

        model = build_resnet50_cosine(num_classes=102, pretrained=True)
        if hasattr(model[1], "s"):
            model[1].s = float(hp["s"])

        ckpt_path = os.path.join(out_dir, f"model_fold{fold}.pth")

        train_one_fold(
            model, train_loader, val_loader,
            lr=float(hp["lr"]),
            weight_decay=float(hp["weight_decay"]),
            label_smoothing=float(hp["label_smoothing"]),
            epochs=epochs, patience=patience,
            ckpt_path=ckpt_path
        )

        model.load_state_dict(torch.load(ckpt_path, map_location=device))
        model.eval()
        criterion = nn.CrossEntropyLoss(label_smoothing=float(hp["label_smoothing"]))
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                total_loss += loss.item()
                correct += (logits.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_loss = total_loss / len(val_loader)
        val_acc = correct / total
        print(f"Fold {fold} validation loss {val_loss:.4f} validation accuracy {val_acc:.4f}")

        # augmented metrics per fold
        train_size = len(train_subset)
        val_size = len(val_subset)
        metrics.append({
            "fold": fold,
            "train_size": train_size,
            "val_size": val_size,
            "val_loss": round(val_loss, 4),
            "val_acc": round(val_acc, 4),
            "ckpt_path": ckpt_path,
            "seed": SEED,
            "experiment_dir": out_dir,
            "hp": {
                "lr": float(hp["lr"]),
                "weight_decay": float(hp["weight_decay"]),
                "s": float(hp["s"]),
                "label_smoothing": float(hp["label_smoothing"]),
                "batch_size": int(hp["batch_size"])
            }
        })

        with open(metrics_path, "w") as f:
            json.dump(metrics, f, indent=2)

    # summary next to metrics.json
    avg_loss = round(float(np.mean([m["val_loss"] for m in metrics])), 4) if metrics else None
    avg_acc  = round(float(np.mean([m["val_acc"] for m in metrics])), 4) if metrics else None
    summary = {
        "experiment_dir": out_dir,
        "folds": len(metrics),
        "avg_val_loss": avg_loss,
        "avg_val_acc": avg_acc,
        "hp": {
            "lr": float(hp["lr"]),
            "weight_decay": float(hp["weight_decay"]),
            "s": float(hp["s"]),
            "label_smoothing": float(hp["label_smoothing"]),
            "batch_size": int(hp["batch_size"])
        },
        "seed": SEED
    }
    with open(os.path.join(out_dir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    print(f"Finished training. Metrics saved to {metrics_path}")
    print(f"Summary saved to {os.path.join(out_dir, 'summary.json')}")
    return metrics

# Run both model sets
baseline_metrics = run_k12_largetrain(OUT_BASELINE, BASELINE_HP, epochs=30, patience=5)
greedy_metrics   = run_k12_largetrain(OUT_GREEDY,   GREEDY_HP,   epochs=30, patience=5)


Starting 12-fold large-train into 12fold_ensemble_ckpt\k12_baseline_largetrain
Fold 1/12 Train 1870 Val 170


C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2157495834.py:48: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=True)
Epoch 1/30:   0%|          | 0/59 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2157495834.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2157495834.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=True):


Epoch 01 | Train 4.1662/0.1465 | Val 3.4666/0.3647
Saved best to 12fold_ensemble_ckpt\k12_baseline_largetrain\model_fold1.pth


Epoch 02 | Train 2.2336/0.7497 | Val 2.1552/0.6941
Saved best to 12fold_ensemble_ckpt\k12_baseline_largetrain\model_fold1.pth


Epoch 03 | Train 1.0632/0.9193 | Val 1.3523/0.8647
Saved best to 12fold_ensemble_ckpt\k12_baseline_largetrain\model_fold1.pth


...

Epoch 12 | Train 1.1352/0.9979 | Val 1.3716/0.9235
EarlyStopping counter 1/5


Epoch 13 | Train 1.1266/0.9984 | Val 1.3697/0.9412
EarlyStopping counter 2/5


Epoch 14 | Train 1.1276/0.9968 | Val 1.4214/0.9059
EarlyStopping counter 3/5


Epoch 15 | Train 1.1241/0.9989 | Val 1.3647/0.9235
EarlyStopping counter 4/5


Epoch 16 | Train 1.1167/1.0000 | Val 1.3609/0.9412
EarlyStopping counter 5/5
Early stopping triggered
Fold 12 validation loss 1.3582 validation accuracy 0.9353
Finished training. Metrics saved to 12fold_ensemble_ckpt\k12_greedy_HP_largetrain\metrics.json
Summary saved to 12fold_ensemble_ckpt\k12_greedy_HP_largetrain\summary.json


## 3.2.7: Baseline 12-fold Largetrain model vs Greedy Optimal Largetrain 12-fold model Largetrain

**Objective:** To evaluate the performance of the 12-fold ensemble model on a large training set and update the results summary.

**Methods:**

- Load the 12-fold model from the large training set and perform average softmax ensemble inference.

- Calculate metrics and write them to a unified JSON file for comparison.

In [9]:
# =============================================================================
# 3.2.7: Inference for large-train k12 model sets and update JSON 
# =============================================================================

import os, json, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

assert 'build_model' in globals(), "build_model must be defined"
assert 'test_loader' in globals(), "test_loader must be defined"
assert 'device' in globals(), "device must be set"

DIR_BASELINE_LT = "12fold_ensemble_ckpt/k12_baseline_largetrain"
DIR_GREEDY_LT   = "12fold_ensemble_ckpt/k12_greedy_HP_largetrain"
OUT_JSON        = os.path.join("12fold_ensemble_ckpt", "12fold_eval_results.json")
NUM_CLASSES     = 102

def _list_ckpts(d):
    return sorted([os.path.join(d, f) for f in os.listdir(d) if f.endswith(".pth")])

def load_models_from_dir(model_dir, num_classes=NUM_CLASSES):
    ckpts = _list_ckpts(model_dir)
    if not ckpts:
        raise FileNotFoundError(f"No .pth files in {model_dir}")
    models = []
    for p in ckpts:
        m = build_resnet50_cosine(num_classes=num_classes, pretrained=True)
        state = torch.load(p, map_location=device)
        m.load_state_dict(state)
        m.to(device).eval()
        models.append(m)
    return models, ckpts

@torch.no_grad()
def ensemble_probs(models, imgs):
    with torch.cuda.amp.autocast(enabled=True):
        acc = None
        for m in models:
            logits = m(imgs)
            probs  = torch.softmax(logits, dim=1)
            acc = probs if acc is None else (acc + probs)
        return (acc / len(models)).detach().cpu().numpy()

def evaluate_dir(model_dir, label_key):
    models, ckpts = load_models_from_dir(model_dir)
    print(f"Evaluating {label_key} using {len(models)} models")

    all_preds, all_labels = [], []
    for imgs, labels in tqdm(test_loader, desc=f"Testing {label_key}"):
        imgs = imgs.to(device, non_blocking=True)
        probs = ensemble_probs(models, imgs)
        preds = probs.argmax(axis=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.numpy().tolist())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1    = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"{label_key} Test Acc {acc:.4f} Macro F1 {macro_f1:.4f} Weighted F1 {weighted_f1:.4f}")

    for m in models:
        del m
    torch.cuda.empty_cache()

    return {
        "num_models": len(ckpts),
        "test_accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
        "models": [p.replace("\\", "/") for p in ckpts]  # normalize paths
    }

results_update = {
    "k12_baseline_largetrain":  evaluate_dir(DIR_BASELINE_LT, "k12_baseline_largetrain"),
    "k12_greedy_HP_largetrain": evaluate_dir(DIR_GREEDY_LT,   "k12_greedy_HP_largetrain"),
}

existing = {}
if os.path.exists(OUT_JSON):
    with open(OUT_JSON, "r") as f:
        try:
            existing = json.load(f)
        except Exception:
            existing = {}

existing.update(results_update)

with open(OUT_JSON, "w") as f:
    json.dump(existing, f, indent=2)

print(f"Saved updated results to {OUT_JSON}")


Evaluating k12_baseline_largetrain using 12 models


Testing k12_baseline_largetrain:   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2357190536.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing k12_baseline_largetrain: 100%|██████████| 97/97 [01:02<00:00,  1.56it/s]


k12_baseline_largetrain Test Acc 0.9545 Macro F1 0.9521 Weighted F1 0.9545
Evaluating k12_greedy_HP_largetrain using 12 models


Testing k12_greedy_HP_largetrain:   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2357190536.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing k12_greedy_HP_largetrain: 100%|██████████| 97/97 [00:59<00:00,  1.62it/s]

k12_greedy_HP_largetrain Test Acc 0.9667 Macro F1 0.9650 Weighted F1 0.9669
Saved updated results to 12fold_ensemble_ckpt\12fold_eval_results.json


## 3.2.8: 12-Fold Model Training with optuna optimal hyperparameters

**Objective**: To perform 12-fold training using the optimal hyperparameters obtained through Optuna search.

**Methods**:

- Read the optimal configuration produced by Optuna.

- Perform deterministic training and early stopping on the 12-fold partition.

- Save the optimal checkpoint and log for each fold.

In [10]:
# =============================================================================
# 3.2.8: 12-Fold Model Training with Optuna Optimal Hyperparameters (deterministic, early stopping)
# =============================================================================

import os, json, random, numpy as np, torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from tqdm import tqdm

# ---------------------------
# Optuna-optimized hyperparameters
# ---------------------------
HP = {
    "lr": 0.00039552247179811825,
    "weight_decay": 5e-5,
    "s": 24.0,
    "label_smoothing": 0.1,
    "batch_size": 16,
    "val_batch_size": 32,
    "epochs": 30,
    "patience": 5,
}

BASE_CKPT_DIR = "12fold_ensemble_ckpt"
SUB_DIR = os.path.join(BASE_CKPT_DIR, "k12_optuna_HP")
METRICS_PATH = os.path.join(SUB_DIR, "metrics_optuna.json")

# ---------------------------
# Determinism setup
# ---------------------------
assert torch.cuda.is_available(), "CUDA is not available."
device = torch.device("cuda:0")

SEED = 1029
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True, warn_only=False)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# ---------------------------
# Trainer (Optuna-tuned HPs)
# ---------------------------
def train_model_hp(model, train_loader, val_loader, *, epochs, lr, weight_decay, label_smoothing, patience, ckpt_path):
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    scaler = GradScaler(enabled=True)

    best_val_loss, patience_counter = float("inf"), 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=True):
                logits = model(imgs)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc  = correct / total

        # ----- Validation -----
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | Train {train_loss:.4f}/{train_acc:.4f} | Val {val_loss:.4f}/{val_acc:.4f}")

        # Early stopping + checkpointing
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), ckpt_path)
            print(f"  Saved best → {ckpt_path}")
        else:
            patience_counter += 1
            print(f"  EarlyStopping counter {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("  Early stopping triggered.")
                break

# ---------------------------
# 12-Fold training using Optuna HPs
# ---------------------------
def run_k12_with_optuna_hp(full_dataset, model_build_fn):
    os.makedirs(SUB_DIR, exist_ok=True)

    # Initialize metrics inside the same subdirectory
    metrics = {"k12_optuna": []}
    if os.path.exists(METRICS_PATH):
        with open(METRICS_PATH, "r") as f:
            try:
                metrics = json.load(f)
            except Exception:
                metrics = {"k12_optuna": []}

    kf = KFold(n_splits=12, shuffle=True, random_state=SEED)
    fold_indices = list(kf.split(range(len(full_dataset))))

    print("\n=== Running 12-Fold training with Optuna-optimized hyperparameters ===")
    for fold, (tr_idx, va_idx) in enumerate(fold_indices, 1):
        print(f"\n[Fold {fold}/12] Train={len(tr_idx)} | Val={len(va_idx)}")

        train_subset = Subset(full_dataset, tr_idx)
        val_subset   = Subset(full_dataset, va_idx)

        g = torch.Generator().manual_seed(SEED + fold)
        train_loader = DataLoader(train_subset, batch_size=HP["batch_size"], shuffle=True,
                                  num_workers=0, pin_memory=True, generator=g)
        val_loader   = DataLoader(val_subset,   batch_size=HP["val_batch_size"], shuffle=False,
                                  num_workers=0, pin_memory=True, generator=g)

        # Build model and set cosine scale s=24 (from Optuna)
        model = model_build_fn()
        if hasattr(model[1], "s"):
            model[1].s = HP["s"]

        fold_ckpt_path = os.path.join(SUB_DIR, f"model_fold{fold}.pth")

        # Train with Optuna-found HPs
        train_model_hp(
            model, train_loader, val_loader,
            epochs=HP["epochs"],
            lr=HP["lr"],
            weight_decay=HP["weight_decay"],
            label_smoothing=HP["label_smoothing"],
            patience=HP["patience"],
            ckpt_path=fold_ckpt_path
        )

        # Quick post-eval on val fold
        model.load_state_dict(torch.load(fold_ckpt_path, map_location=device))
        model.eval()
        criterion = nn.CrossEntropyLoss(label_smoothing=HP["label_smoothing"])
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                total_loss += loss.item()
                correct += (logits.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_loss = total_loss / len(val_loader)
        val_acc  = correct / total
        print(f"  [Fold {fold}] Val Loss={val_loss:.4f} | Val Acc={val_acc:.4f}")

        metrics["k12_optuna"].append({
            "fold": fold,
            "val_loss": round(val_loss, 4),
            "val_acc": round(val_acc, 4),
            "ckpt_path": fold_ckpt_path,
            "hp": {k: (float(v) if isinstance(v, (int, float)) else v) for k, v in HP.items()}
        })

        with open(METRICS_PATH, "w") as f:
            json.dump(metrics, f, indent=2)

    print(f"\n=== 12-fold Optuna-trained models complete. Metrics → {METRICS_PATH} ===")
    return metrics

# ---- Run it (build_model must exist) ----
run_k12_with_optuna_hp(
    full_dataset=train_loader.dataset,
    model_build_fn=lambda: build_resnet50_cosine(num_classes=102, pretrained=True)
)



=== Running 12-Fold training with Optuna-optimized hyperparameters ===

[Fold 1/12] Train=935 | Val=85


C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2861917886.py:61: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=True)
Epoch 1/30:   0%|          | 0/59 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2861917886.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\2861917886.py:90: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=True):


Epoch 01 | Train 4.0066/0.2139 | Val 3.0183/0.4824
  Saved best → 12fold_ensemble_ckpt\k12_optuna_HP\model_fold1.pth


Epoch 02 | Train 2.0939/0.7198 | Val 1.9601/0.7294
  Saved best → 12fold_ensemble_ckpt\k12_optuna_HP\model_fold1.pth


Epoch 03 | Train 1.3067/0.9176 | Val 1.5361/0.8000
  Saved best → 12fold_ensemble_ckpt\k12_optuna_HP\model_fold1.pth


...

Epoch 13 | Train 0.8428/0.9957 | Val 1.2116/0.8941
  EarlyStopping counter 2/5


Epoch 14 | Train 0.8652/0.9925 | Val 1.2284/0.8824
  EarlyStopping counter 3/5


Epoch 15 | Train 0.8367/0.9936 | Val 1.2377/0.8471
  EarlyStopping counter 4/5


Epoch 16 | Train 0.8160/0.9989 | Val 1.1637/0.8941
  EarlyStopping counter 5/5
  Early stopping triggered.
  [Fold 12] Val Loss=1.1646 | Val Acc=0.8824

=== 12-fold Optuna-trained models complete. Metrics → 12fold_ensemble_ckpt\k12_optuna_HP\metrics_optuna.json ===


{'k12_optuna': [{'fold': 1,
   'val_loss': 1.2553,
   'val_acc': 0.8824,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_optuna_HP\\model_fold1.pth',
   'hp': {'lr': 0.00039552247179811825,
    'weight_decay': 5e-05,
    's': 24.0,
    'label_smoothing': 0.1,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
    'patience': 5.0}},
  {'fold': 2,
   'val_loss': 1.1268,
   'val_acc': 0.9294,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_optuna_HP\\model_fold2.pth',
   'hp': {'lr': 0.00039552247179811825,
    'weight_decay': 5e-05,
    's': 24.0,
    'label_smoothing': 0.1,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
    'patience': 5.0}},
  {'fold': 3,
   'val_loss': 1.1802,
   'val_acc': 0.8941,
   'ckpt_path': '12fold_ensemble_ckpt\\k12_optuna_HP\\model_fold3.pth',
   'hp': {'lr': 0.00039552247179811825,
    'weight_decay': 5e-05,
    's': 24.0,
    'label_smoothing': 0.1,
    'batch_size': 16.0,
    'val_batch_size': 32.0,
    'epochs': 30.0,
 

## 3.2.9: 12-fold Optuna Optimal Model inference

**Objective:** To evaluate the performance of the 12-fold ensemble under Optuna hyperparameters.
**Method:**
- Load the best 12-fold model and perform average softmax ensemble inference on the test set.
- Record accuracy and macro/weighted metrics to a JSON summary.

In [11]:
# =============================================================================
# 3.2.9. Evaluate k12_optuna_HP (averaged softmax)
# =============================================================================
import os, json, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

assert 'build_model' in globals(), "build_model(...) must be defined above."
assert 'test_loader' in globals(), "test_loader must exist."
assert 'device' in globals(), "device must be set."

DIR_OPTUNA  = "12fold_ensemble_ckpt/k12_optuna_HP"
OUT_JSON    = os.path.join("12fold_ensemble_ckpt", "12fold_eval_results.json")
NUM_CLASSES = 102

def load_models_from_dir(model_dir, num_classes=NUM_CLASSES):
    ckpts = sorted([os.path.join(model_dir, f) for f in os.listdir(model_dir) if f.endswith(".pth")])
    if not ckpts:
        raise FileNotFoundError(f"No .pth files in {model_dir}")
    models = []
    for p in ckpts:
        m = build_resnet50_cosine(num_classes=num_classes, pretrained=True)
        state = torch.load(p, map_location=device)
        m.load_state_dict(state)  # saved with torch.save(model.state_dict(), path)
        m.to(device).eval()
        models.append(m)
    return models, ckpts

@torch.no_grad()
def ensemble_probs(models, imgs):
    with torch.cuda.amp.autocast(enabled=True):
        acc = None
        for m in models:
            logits = m(imgs)
            probs  = torch.softmax(logits, dim=1)
            acc = probs if acc is None else (acc + probs)
        return (acc / len(models)).detach().cpu().numpy()

def evaluate_dir(model_dir, label):
    models, ckpts = load_models_from_dir(model_dir)
    print(f"\n=== Evaluating {label} ===")
    print(f"Loaded {len(models)} models from: {model_dir}")

    all_preds, all_labels = [], []
    for imgs, labels in tqdm(test_loader, desc=f"Testing ({label})"):
        imgs = imgs.to(device, non_blocking=True)
        probs = ensemble_probs(models, imgs)
        preds = probs.argmax(axis=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.numpy().tolist())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1    = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"[{label}] Test Acc: {acc:.4f} | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f}")

    # free GPU
    for m in models:
        del m
    torch.cuda.empty_cache()

    return {
        "num_models": len(ckpts),
        "test_accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
        "models": ckpts,  # paths for traceability
    }

# Run only on k12_optuna_HP
optuna_result = evaluate_dir(DIR_OPTUNA, "k12_optuna_HP")

# Load existing results JSON (if any), update with optuna result, and save back
results = {}
if os.path.exists(OUT_JSON):
    try:
        with open(OUT_JSON, "r") as f:
            results = json.load(f)
    except Exception:
        results = {}

results["k12_optuna_HP"] = optuna_result

os.makedirs(os.path.dirname(OUT_JSON), exist_ok=True)
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\nSaved optuna evaluation to {OUT_JSON}")



=== Evaluating k12_optuna_HP ===
Loaded 12 models from: 12fold_ensemble_ckpt/k12_optuna_HP


Testing (k12_optuna_HP):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3074246008.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing (k12_optuna_HP): 100%|██████████| 97/97 [00:59<00:00,  1.62it/s]

[k12_optuna_HP] Test Acc: 0.9436 | Macro F1: 0.9422 | Weighted F1: 0.9439

Saved optuna evaluation to 12fold_ensemble_ckpt\12fold_eval_results.json


## 3.2.10: Train 12-fold large-train on Optuna hyperparameters

**Objective**: To train 12-fold ensembles using Optuna hyperparameters on a merged large training set.
**Method**:
- Perform 12-fold training on the merged data (deterministic + early stopping or code-defined strategy).
- Output the optimal weights and logs for each fold.

In [12]:
# =============================================================================
# 3.2.10. Train 12-fold large-train models (Optuna HP only)
# =============================================================================
import os, json, torch, numpy as np
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from tqdm import tqdm

# Requirements from earlier sections
assert "combined_dataset" in globals(), "combined_dataset must be defined from Section 6C"
assert "build_model" in globals(), "build_model must be defined above"
assert "device" in globals(), "device must be set above"

SEED = 1029
BASE_DIR = "12fold_ensemble_ckpt"
OUT_OPTUNA = os.path.join(BASE_DIR, "k12_optuna_HP_largetrain")
os.makedirs(OUT_OPTUNA, exist_ok=True)

# Optuna-found hyperparameters
OPTUNA_HP = {
    "lr": 0.00039552247179811825,
    "weight_decay": 5e-05,
    "s": 24.0,
    "label_smoothing": 0.1,
    "batch_size": 16
}

def train_one_fold(model, train_loader, val_loader, *, lr, weight_decay, label_smoothing, epochs=30, patience=5, ckpt_path):
    os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2)
    scaler = GradScaler(enabled=True)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=True):
                logits = model(imgs)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        scheduler.step(val_loss)

        print(f"Epoch {epoch:02d} | Train {train_loss:.4f}/{train_acc:.4f} | Val {val_loss:.4f}/{val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), ckpt_path)
            print(f"Saved best to {ckpt_path}")
        else:
            patience_counter += 1
            print(f"EarlyStopping counter {patience_counter}/5")
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

def run_k12_largetrain(out_dir, hp, *, epochs=30, patience=5):
    metrics_path = os.path.join(out_dir, "metrics.json")
    metrics = []

    kf = KFold(n_splits=12, shuffle=True, random_state=SEED)
    indices = np.arange(len(combined_dataset))

    print(f"Starting 12-fold large-train into {out_dir}")
    for fold, (train_idx, val_idx) in enumerate(kf.split(indices), 1):
        print(f"Fold {fold}/12 Train {len(train_idx)} Val {len(val_idx)}")

        train_subset = Subset(combined_dataset, train_idx.tolist())
        val_subset   = Subset(combined_dataset, val_idx.tolist())

        g = torch.Generator().manual_seed(SEED + fold)
        train_loader = DataLoader(
            train_subset, batch_size=hp["batch_size"], shuffle=True,
            num_workers=0, pin_memory=True, generator=g
        )
        val_loader = DataLoader(
            val_subset, batch_size=64, shuffle=False,
            num_workers=0, pin_memory=True, generator=g
        )

        model = build_resnet50_cosine(num_classes=102, pretrained=True)
        if hasattr(model[1], "s"):
            model[1].s = float(hp["s"])

        ckpt_path = os.path.join(out_dir, f"model_fold{fold}.pth")

        train_one_fold(
            model, train_loader, val_loader,
            lr=float(hp["lr"]),
            weight_decay=float(hp["weight_decay"]),
            label_smoothing=float(hp["label_smoothing"]),
            epochs=epochs, patience=patience,
            ckpt_path=ckpt_path
        )

        model.load_state_dict(torch.load(ckpt_path, map_location=device))
        model.eval()
        criterion = nn.CrossEntropyLoss(label_smoothing=float(hp["label_smoothing"]))
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad(), autocast(enabled=True):
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                total_loss += loss.item()
                correct += (logits.argmax(1) == labels).sum().item()
                total += labels.size(0)

        val_loss = total_loss / len(val_loader)
        val_acc = correct / total
        print(f"Fold {fold} validation loss {val_loss:.4f} validation accuracy {val_acc:.4f}")

        metrics.append({
            "fold": fold,
            "train_size": len(train_subset),
            "val_size": len(val_subset),
            "val_loss": round(val_loss, 4),
            "val_acc": round(val_acc, 4),
            "ckpt_path": ckpt_path,
            "seed": SEED,
            "experiment_dir": out_dir,
            "hp": {
                "lr": float(hp["lr"]),
                "weight_decay": float(hp["weight_decay"]),
                "s": float(hp["s"]),
                "label_smoothing": float(hp["label_smoothing"]),
                "batch_size": int(hp["batch_size"])
            }
        })

        with open(metrics_path, "w") as f:
            json.dump(metrics, f, indent=2)

    avg_loss = round(float(np.mean([m["val_loss"] for m in metrics])), 4) if metrics else None
    avg_acc  = round(float(np.mean([m["val_acc"] for m in metrics])), 4) if metrics else None
    summary = {
        "experiment_dir": out_dir,
        "folds": len(metrics),
        "avg_val_loss": avg_loss,
        "avg_val_acc": avg_acc,
        "hp": {
            "lr": float(hp["lr"]),
            "weight_decay": float(hp["weight_decay"]),
            "s": float(hp["s"]),
            "label_smoothing": float(hp["label_smoothing"]),
            "batch_size": int(hp["batch_size"])
        },
        "seed": SEED
    }
    with open(os.path.join(out_dir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    print(f"Finished training. Metrics saved to {metrics_path}")
    print(f"Summary saved to {os.path.join(out_dir, 'summary.json')}")
    return metrics

# Train only the Optuna configuration
optuna_metrics = run_k12_largetrain(OUT_OPTUNA, OPTUNA_HP, epochs=30, patience=5)



Starting 12-fold large-train into 12fold_ensemble_ckpt\k12_optuna_HP_largetrain
Fold 1/12 Train 1870 Val 170


C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3655634412.py:39: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=True)
Epoch 1/30:   0%|          | 0/117 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3655634412.py:51: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True):
C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\3655634412.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast(enabled=True):


Epoch 01 | Train 3.1325/0.4289 | Val 1.7058/0.8059
Saved best to 12fold_ensemble_ckpt\k12_optuna_HP_largetrain\model_fold1.pth


Epoch 02 | Train 1.3146/0.8813 | Val 1.2198/0.8824
Saved best to 12fold_ensemble_ckpt\k12_optuna_HP_largetrain\model_fold1.pth


Epoch 03 | Train 1.0082/0.9631 | Val 1.1914/0.9000
Saved best to 12fold_ensemble_ckpt\k12_optuna_HP_largetrain\model_fold1.pth


...

Epoch 22 | Train 0.7879/1.0000 | Val 1.0861/0.9294
EarlyStopping counter 1/5


Epoch 23 | Train 0.7892/0.9989 | Val 1.0835/0.9235
EarlyStopping counter 2/5


Epoch 24 | Train 0.7876/1.0000 | Val 1.1207/0.9176
EarlyStopping counter 3/5


Epoch 25 | Train 0.7867/1.0000 | Val 1.0920/0.9235
EarlyStopping counter 4/5


Epoch 26 | Train 0.7862/1.0000 | Val 1.1230/0.9118
EarlyStopping counter 5/5
Early stopping triggered
Fold 12 validation loss 1.0877 validation accuracy 0.9353
Finished training. Metrics saved to 12fold_ensemble_ckpt\k12_optuna_HP_largetrain\metrics.json
Summary saved to 12fold_ensemble_ckpt\k12_optuna_HP_largetrain\summary.json


## 3.2.11: 12-fold Optuna Optimal Largetrain Model inference

**Objective**: To evaluate the ensemble performance of a 12-fold ensemble with a large training set and Optuna hyperparameters.
**Method**:
- Load the model for each fold and average the softmax to obtain the final probability.
- Calculate the metrics and summarize them in JSON.

In [13]:
# =============================================================================
# 3.2.11. Evaluate k12_optuna_HP_largetrain (averaged softmax)
# =============================================================================
import os, json, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

assert 'build_model' in globals(), "build_model(...) must be defined above."
assert 'test_loader' in globals(), "test_loader must exist."
assert 'device' in globals(), "device must be set."

DIR_OPTUNA  = "12fold_ensemble_ckpt/k12_optuna_HP_largetrain"
OUT_JSON    = os.path.join("12fold_ensemble_ckpt", "12fold_eval_results.json")
NUM_CLASSES = 102

def load_models_from_dir(model_dir, num_classes=NUM_CLASSES):
    ckpts = sorted([os.path.join(model_dir, f) for f in os.listdir(model_dir) if f.endswith(".pth")])
    if not ckpts:
        raise FileNotFoundError(f"No .pth files in {model_dir}")
    models = []
    for p in ckpts:
        m = build_resnet50_cosine(num_classes=num_classes, pretrained=True)
        state = torch.load(p, map_location=device)
        m.load_state_dict(state)  # saved with torch.save(model.state_dict(), path)
        m.to(device).eval()
        models.append(m)
    return models, ckpts

@torch.no_grad()
def ensemble_probs(models, imgs):
    with torch.cuda.amp.autocast(enabled=True):
        acc = None
        for m in models:
            logits = m(imgs)
            probs  = torch.softmax(logits, dim=1)
            acc = probs if acc is None else (acc + probs)
        return (acc / len(models)).detach().cpu().numpy()

def evaluate_dir(model_dir, label):
    models, ckpts = load_models_from_dir(model_dir)
    print(f"\n=== Evaluating {label} ===")
    print(f"Loaded {len(models)} models from: {model_dir}")

    all_preds, all_labels = [], []
    for imgs, labels in tqdm(test_loader, desc=f"Testing ({label})"):
        imgs = imgs.to(device, non_blocking=True)
        probs = ensemble_probs(models, imgs)
        preds = probs.argmax(axis=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.numpy().tolist())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1    = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"[{label}] Test Acc: {acc:.4f} | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f}")

    # free GPU
    for m in models:
        del m
    torch.cuda.empty_cache()

    return {
        "num_models": len(ckpts),
        "test_accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
        "models": ckpts,  # paths for traceability
    }

# Run only on k12_optuna_HP_largetrain
optuna_result = evaluate_dir(DIR_OPTUNA, "k12_optuna_HP_largetrain")

# Load existing results JSON (if any), update with optuna result, and save back
results = {}
if os.path.exists(OUT_JSON):
    try:
        with open(OUT_JSON, "r") as f:
            results = json.load(f)
    except Exception:
        results = {}

results["k12_optuna_HP_largetrain"] = optuna_result

os.makedirs(os.path.dirname(OUT_JSON), exist_ok=True)
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\nSaved optuna evaluation to {OUT_JSON}")



=== Evaluating k12_optuna_HP_largetrain ===
Loaded 12 models from: 12fold_ensemble_ckpt/k12_optuna_HP_largetrain


Testing (k12_optuna_HP_largetrain):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\686783902.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
Testing (k12_optuna_HP_largetrain): 100%|██████████| 97/97 [01:02<00:00,  1.56it/s]

[k12_optuna_HP_largetrain] Test Acc: 0.9689 | Macro F1: 0.9672 | Weighted F1: 0.9692

Saved optuna evaluation to 12fold_ensemble_ckpt\12fold_eval_results.json


## 3.4.1: Siamese few shot training on 12 fold optuna hyperparameter
This section evaluates the 5-shot Siamese network using a 12-fold ensemble, where each fold produces an independently trained similarity-based classifier. The goal is to assess whether the Siamese architecture can generalize effectively under low-data conditions and whether ensembling improves stability.
Training Setup
Each fold trains a Siamese model using:
- a shared ResNet-50 backbone
- a projection head that maps features to a 512-dimensional embedding
- a contrastive loss function that pulls together embeddings of similar pairs and pushes apart embeddings of dissimilar pairs

Prototype Construction
For each fold, class prototypes are computed by averaging normalized embeddings of all training images belonging to each class. These prototypes serve as the reference points for 5-shot classification.

Inference and Model Voting
At test time, each image is processed by all 12 models. Each model predicts the most similar class by comparing its query embedding with the class prototypes using dot-product similarity. The final prediction is determined through model voting, with deterministic probability-based tie-breakers:
- majority vote,
- highest mean probability among tied classes,
- highest max probability among tied classes,
- smallest class index (final fallback).

This approach evaluates the Siamese few-shot framework under a robust 12-model ensemble, measuring how effectively similarity-based learning can generalize to unseen flower images.

In [14]:
# 3.4.1.A Few-shot helper functions
from collections import defaultdict
import random

def build_class_index(dataset):
    """Return {class_id: [indices]} mapping for a given dataset."""
    by_cls = defaultdict(list)
    for idx, (_, y) in enumerate(dataset):
        by_cls[int(y)].append(idx)
    return by_cls

def build_pairs(index_by_class, k_per_class, pairs_per_class, rng):
    """
    Build balanced positive/negative pairs for Siamese training.
    index_by_class: dict {class: [indices]}
    k_per_class: number of examples per class to draw
    pairs_per_class: number of positive pairs per class
    rng: random.Random instance (for determinism)
    """
    # choose k per class
    chosen_per_class = {
        c: rng.sample(idxs, min(k_per_class, len(idxs)))
        for c, idxs in index_by_class.items()
    }

    # positive pairs
    pos_pairs = []
    for c, idxs in chosen_per_class.items():
        cand = []
        for i in range(len(idxs)):
            for j in range(i + 1, len(idxs)):
                cand.append((idxs[i], idxs[j]))
        rng.shuffle(cand)
        pos_pairs.extend([(a, b, 1) for (a, b) in cand[:pairs_per_class]])

    # negative pairs (balanced)
    all_classes = list(chosen_per_class.keys())
    neg_pairs = []
    for c in all_classes:
        a_list = rng.choices(chosen_per_class[c], k=pairs_per_class)
        for a in a_list:
            c2 = rng.choice([x for x in all_classes if x != c])
            b = rng.choice(chosen_per_class[c2])
            neg_pairs.append((a, b, 0))

    pairs = pos_pairs + neg_pairs
    rng.shuffle(pairs)
    return pairs, chosen_per_class

class PairDataset(torch.utils.data.Dataset):
    """Dataset of image pairs for contrastive (Siamese) training."""
    def __init__(self, base_dataset, pairs):
        self.base = base_dataset
        self.pairs = pairs
        self.tf = base_dataset.transform

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, i):
        i1, i2, y = self.pairs[i]
        x1, _ = self.base[i1]
        x2, _ = self.base[i2]
        return x1, x2, torch.tensor(y, dtype=torch.float32) 
    
# === Siamese Network & Contrastive Loss Definitions ===
import torch
import torch.nn as nn
import torch.nn.functional as F

# Small projection head applied after the frozen backbone
class ProjectionHead(nn.Module):
    def __init__(self, in_dim=2048, out_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, out_dim)
        )
    def forward(self, x):
        z = self.net(x)
        return F.normalize(z, dim=1)

# Siamese network wrapper: shared backbone + projection head
class Siamese(nn.Module):
    def __init__(self, backbone, proj_out=512, freeze_backbone=True):
        super().__init__()
        self.backbone = backbone  # expects [B,3,224,224] -> [B,2048]
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
        self.proj = ProjectionHead(2048, proj_out)

    def embed(self, x):
        feats = self.backbone(x)     # [B,2048]
        return self.proj(feats)      # [B,proj_out], L2-normalized

    def forward(self, x1, x2):
        z1 = self.embed(x1)
        z2 = self.embed(x2)
        sim = (z1 * z2).sum(dim=1)   # cosine similarity (for monitoring)
        return z1, z2, sim

# Contrastive loss (Hadsell et al., 2006)
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
    def forward(self, z1, z2, y):
        d = F.pairwise_distance(z1, z2)
        pos = y * (d ** 2)
        neg = (1 - y) * (F.relu(self.margin - d) ** 2)
        return (pos + neg).mean()



In [15]:
# 3.4.1.B: 12-Fold Siamese Few-Shot (5-shot) ===
# - 12 folds
# - 5-shot training/eval
# - 10 epochs max with EarlyStopping(patience=3)
# - num_workers=0 baseline
# - Save ONLY the best per fold as: model_fold{fold}.pth

import os, time, math, json, random, numpy as np, torch, torch.nn.functional as F
from torch.utils.data import Subset, DataLoader, Dataset
from sklearn.model_selection import KFold
from collections import defaultdict
from tqdm import tqdm

# Preconditions
assert "train_loader" in globals() and "val_loader" in globals(), "Require train_loader and val_loader"
assert "Siamese" in globals(), "Siamese must be defined"
assert "ContrastiveLoss" in globals(), "ContrastiveLoss must be defined"
assert "build_model" in globals(), "build_model must be defined"

# GPU
assert torch.cuda.is_available(), "CUDA GPU not available."
device = torch.device("cuda")
torch.backends.cudnn.benchmark = True
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

SEED = 1029
BASE_DIR = "12fold_ensemble_ckpt"
os.makedirs(BASE_DIR, exist_ok=True)

# Optional warm-start backbone dir (keep None if not using)
BACKBONE_DIR = None

# Optuna HPs for the projection head optimizer
OPTUNA_HP = {
    "lr": 3.9552247179811825e-04,
    "weight_decay": 5e-05,
    "s": 24.0,               # not used by Siamese projection
    "label_smoothing": 0.1,  # not used here
    "batch_size": 16,
}

# Training knobs
PAIRS_PER_CLASS    = 40
EPOCHS_SIAMESE     = 10
EARLY_PATIENCE     = 3
BATCH_SIZE_PAIRS   = 64
NUM_WORKERS        = 0
PIN_MEMORY         = True
PERSISTENT_WORKERS = False
MARGIN             = 1.0
NUM_CLASSES        = 102

# Episodic evaluation (5-shot)
N_WAY               = 5
K_SHOT              = 5           # 5-shot
Q_PER_CLASS         = 5
EPISODES_EVAL_FULL  = 100         # per-fold final report
EPISODES_EVAL_EPOCH = 30          # quick per-epoch val for early stopping

# ----- FAST label index builders (no image decode) -----
def fast_build_class_index_any(dataset):
    """
    Returns dict: class_id -> list of indices valid for *dataset*.
    Works for Flowers102 and Subset(Flowers102) without touching images.
    """
    by_cls = defaultdict(list)
    if isinstance(dataset, Subset):
        base = dataset.dataset
        idxs = dataset.indices
        labels = getattr(base, "_labels", None)
        assert labels is not None, "Flowers102 labels not found on base dataset."
        for i_sub, i_base in enumerate(idxs):
            c = int(labels[i_base])
            by_cls[c].append(i_sub)
    else:
        labels = getattr(dataset, "_labels", None)
        assert labels is not None, "Flowers102 labels not found on dataset."
        for i in range(len(labels)):
            by_cls[int(labels[i])].append(i)
    return by_cls

def fast_build_pairs(index_by_class, k_per_class, pairs_per_class, rng):
    """Build balanced pos/neg pairs using indices only (no image access)."""
    chosen_per_class = {c: rng.sample(idxs, min(k_per_class, len(idxs)))
                        for c, idxs in index_by_class.items()}

    pos_pairs = []
    for c, idxs in chosen_per_class.items():
        # all unique intra-class pairs
        cand = []
        for i in range(len(idxs)):
            for j in range(i + 1, len(idxs)):
                cand.append((idxs[i], idxs[j]))
        rng.shuffle(cand)
        pos_pairs.extend([(a, b, 1) for (a, b) in cand[:pairs_per_class]])

    all_classes = list(chosen_per_class.keys())
    neg_pairs = []
    for c in all_classes:
        if len(chosen_per_class[c]) == 0:
            continue
        a_list = rng.choices(chosen_per_class[c], k=pairs_per_class)  # with replacement is fine
        valid_others = [x for x in all_classes if x != c and len(chosen_per_class[x]) > 0]
        if not valid_others:
            continue
        for a in a_list:
            c2 = rng.choice(valid_others)
            b = rng.choice(chosen_per_class[c2])
            neg_pairs.append((a, b, 0))

    pairs = pos_pairs + neg_pairs
    rng.shuffle(pairs)
    return pairs, chosen_per_class

# Subset-safe PairDataset
class PairDataset(Dataset):
    def __init__(self, base_dataset, pairs):
        self.base = base_dataset
        self.pairs = pairs
    def __len__(self): return len(self.pairs)
    def __getitem__(self, idx):
        i1, i2, y = self.pairs[idx]
        x1, _ = self.base[i1]
        x2, _ = self.base[i2]
        return x1, x2, torch.tensor(y, dtype=torch.float32)

@torch.no_grad()
def episodic_eval(backbone, proj, base_dataset, n_way=5, k_shot=5, q_per_class=5, episodes=100, seed=SEED):
    rng = random.Random(seed)
    torch.manual_seed(seed); np.random.seed(seed)
    backbone.eval(); proj.eval()

    by_cls = fast_build_class_index_any(base_dataset)
    classes = list(by_cls.keys())
    accs = []

    def load_batch(pairs):
        xs, ys = [], []
        for i, c in pairs:
            x, _ = base_dataset[i]
            xs.append(x); ys.append(c)
        xs = torch.stack(xs).to(device, non_blocking=True)
        ys = torch.tensor(ys, device=device)
        return xs, ys

    for _ in range(episodes):
        episode_classes = rng.sample(classes, n_way)
        support_idx, query_idx = [], []
        for c in episode_classes:
            idxs = by_cls[c]
            need = k_shot + q_per_class
            sampled = rng.choices(idxs, k=need) if len(idxs) < need else rng.sample(idxs, k=need)
            support_idx.extend([(i, c) for i in sampled[:k_shot]])
            query_idx.extend([(i, c) for i in sampled[k_shot:]])

        xs_s, ys_s = load_batch(support_idx)
        xs_q, ys_q = load_batch(query_idx)

        with torch.amp.autocast("cuda", dtype=torch.float16):
            z_s = F.normalize(proj(backbone(xs_s)), dim=1)
            z_q = F.normalize(proj(backbone(xs_q)), dim=1)
            protos = torch.stack([z_s[(ys_s == c)].mean(dim=0) for c in episode_classes])
            sims = z_q @ protos.t()
            preds = sims.argmax(dim=1)

        class_to_pos = {c: i for i, c in enumerate(episode_classes)}
        ys_true = torch.tensor([class_to_pos[int(c)] for c in ys_q.tolist()], device=device)
        accs.append((preds == ys_true).float().mean().item())

    mean = float(np.mean(accs))
    se = float(np.std(accs) / math.sqrt(len(accs)))
    return mean, se

def train_12fold_siamese_k5():
    key_name  = "k5_optuna_siamese_5shot_12fold"
    out_dir   = os.path.join(BASE_DIR, "12fold_optuna_siamese_5shot")
    os.makedirs(out_dir, exist_ok=True)
    metrics_path = os.path.join(out_dir, "metrics.json")

    base_train = train_loader.dataset
    base_val   = val_loader.dataset

    # load/init metrics.json
    if os.path.exists(metrics_path):
        try:
            with open(metrics_path, "r") as f: metrics_blob = json.load(f)
        except Exception:
            metrics_blob = {}
    else:
        metrics_blob = {}
    metrics_blob.setdefault(key_name, [])

    n_splits = 12
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    indices = np.arange(len(base_train))

    print(f"Starting {n_splits}-fold Siamese training for K=5 (10 epochs max, patience=3, workers=0)")
    for fold, (tr_idx, _held) in enumerate(kf.split(indices), 1):
        print(f"[Fold {fold}/{n_splits}] Building pairs from train split (no image decode for labels)")
        train_subset = Subset(base_train, tr_idx.tolist())

        rng = random.Random(SEED + fold * 37 + 5)
        by_cls = fast_build_class_index_any(train_subset)
        pairs, _ = fast_build_pairs(by_cls, k_per_class=K_SHOT, pairs_per_class=PAIRS_PER_CLASS, rng=rng)

        pair_ds = PairDataset(train_subset, pairs)
        pair_loader = DataLoader(
            pair_ds,
            batch_size=BATCH_SIZE_PAIRS,
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            persistent_workers=PERSISTENT_WORKERS
        )
        print(f"[Fold {fold}] PairDataset size: {len(pair_ds)} (pos≈neg)")

        # Backbone from supervised definition; freeze
        model_full = build_model(num_classes=NUM_CLASSES, pretrained=True)
        backbone = model_full[0].to(device)

        if BACKBONE_DIR:
            ckpt_sup = os.path.join(BACKBONE_DIR, f"model_fold{fold}.pth")
            if os.path.exists(ckpt_sup):
                try:
                    state = torch.load(ckpt_sup, map_location=device)
                    backbone.load_state_dict(state, strict=False)
                    print(f"[Fold {fold}] Loaded supervised backbone from {ckpt_sup}")
                except Exception as e:
                    print(f"[Fold {fold}] Warning: could not load backbone from {ckpt_sup}: {e}")

        for p in backbone.parameters():
            p.requires_grad = False

        siam = Siamese(backbone, proj_out=512, freeze_backbone=True).to(device)
        criterion = ContrastiveLoss(MARGIN).to(device)
        optimizer = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, siam.parameters()),
            lr=OPTUNA_HP["lr"],
            weight_decay=OPTUNA_HP["weight_decay"]
        )
        scaler = torch.amp.GradScaler("cuda")

        ckpt_best = os.path.join(out_dir, f"model_fold{fold}.pth")
        print(f"[Fold {fold}] Will save BEST -> {ckpt_best}")

        best_acc = -1.0
        patience_counter = 0

        for epoch in range(1, EPOCHS_SIAMESE + 1):
            t0 = time.time()
            siam.train()
            running, n = 0.0, 0

            for x1, x2, y in tqdm(pair_loader, desc=f"[Fold {fold}] Epoch {epoch}/{EPOCHS_SIAMESE}", leave=False):
                x1 = x1.to(device, non_blocking=True)
                x2 = x2.to(device, non_blocking=True)
                y  = y.to(device, non_blocking=True)

                optimizer.zero_grad(set_to_none=True)
                with torch.amp.autocast("cuda", dtype=torch.float16):
                    z1, z2, _ = siam(x1, x2)
                    loss = criterion(z1, z2, y)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                running += loss.item() * x1.size(0)
                n += x1.size(0)

            epoch_time = time.time() - t0
            print(f"[Fold {fold}] Epoch {epoch} train contrastive loss {running / max(1, n):.4f} | time {epoch_time:.2f}s")

            # Quick episodic eval for early stopping
            mean_acc_epoch, _ = episodic_eval(
                backbone=siam.backbone,
                proj=siam.proj,
                base_dataset=base_val,
                n_way=N_WAY,
                k_shot=K_SHOT,
                q_per_class=Q_PER_CLASS,
                episodes=EPISODES_EVAL_EPOCH,
                seed=SEED + fold + epoch
            )
            print(f"[Fold {fold}] Epoch {epoch} quick VAL (k=5) acc {mean_acc_epoch*100:.2f}%")

            if mean_acc_epoch > best_acc:
                best_acc = mean_acc_epoch
                patience_counter = 0
                torch.save({"proj": siam.proj.state_dict()}, ckpt_best)
                print(f"[Fold {fold}] New BEST saved -> {ckpt_best}")
            else:
                patience_counter += 1
                print(f"[Fold {fold}] EarlyStopping counter {patience_counter}/{EARLY_PATIENCE}")
                if patience_counter >= EARLY_PATIENCE:
                    print(f"[Fold {fold}] Early stopping.")
                    break

        # Load BEST for final episodic VAL
        best_state = torch.load(ckpt_best, map_location=device) if os.path.exists(ckpt_best) else None
        if best_state is not None:
            siam.proj.load_state_dict(best_state["proj"])

        mean_acc, se = episodic_eval(
            backbone=siam.backbone,
            proj=siam.proj,
            base_dataset=base_val,
            n_way=N_WAY,
            k_shot=K_SHOT,
            q_per_class=Q_PER_CLASS,
            episodes=EPISODES_EVAL_FULL,
            seed=SEED + fold
        )
        print(f"[Fold {fold}] FINAL {N_WAY}-way {K_SHOT}-shot VAL: {mean_acc*100:.2f}% ± {se*100:.2f}%")

        # Log BEST metrics
        # metrics.json kept for traceability (optional)
        metrics_entry = {
            "fold": fold,
            "val_loss": None,
            "val_acc": round(mean_acc, 4),
            "val_stderr": round(se, 4),
            "ckpt_path": (ckpt_best.replace("/", "\\") if os.name == "nt" else ckpt_best),
            "hp": {
                "lr": float(OPTUNA_HP["lr"]),
                "weight_decay": float(OPTUNA_HP["weight_decay"]),
                "s": float(OPTUNA_HP["s"]),
                "label_smoothing": float(OPTUNA_HP["label_smoothing"]),
                "batch_size": float(OPTUNA_HP["batch_size"]),
                "val_batch_size": float(BATCH_SIZE_PAIRS),
                "epochs": float(EPOCHS_SIAMESE),
                "patience": float(EARLY_PATIENCE)
            }
        }
        metrics_blob[key_name].append(metrics_entry)
        with open(metrics_path, "w") as f:
            json.dump(metrics_blob, f, indent=2)

        # Clean up
        del siam, model_full, backbone, pair_loader, pair_ds
        torch.cuda.empty_cache()

    print(f"Completed {n_splits}-fold Siamese K=5. Metrics -> {metrics_path}")

# Run it
train_12fold_siamese_k5()


Using GPU: NVIDIA GeForce RTX 3080
Starting 12-fold Siamese training for K=5 (10 epochs max, patience=3, workers=0)
[Fold 1/12] Building pairs from train split (no image decode for labels)
[Fold 1] PairDataset size: 5100 (pos≈neg)
[Fold 1] Will save BEST -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\model_fold1.pth


[Fold 1] Epoch 1 train contrastive loss 0.1157 | time 102.75s
[Fold 1] Epoch 1 quick VAL (k=5) acc 88.80%
[Fold 1] New BEST saved -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\model_fold1.pth


[Fold 1] Epoch 2 train contrastive loss 0.0756 | time 101.74s
[Fold 1] Epoch 2 quick VAL (k=5) acc 88.40%
[Fold 1] EarlyStopping counter 1/3


[Fold 1] Epoch 3 train contrastive loss 0.0612 | time 102.23s
[Fold 1] Epoch 3 quick VAL (k=5) acc 93.73%
[Fold 1] New BEST saved -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\model_fold1.pth


[Fold 1] Epoch 4 train contrastive loss 0.0531 | time 103.23s
[Fold 1] Epoch 4 quick VAL (k=5) acc 92.13%
[Fold 1] EarlyStopping counter 1/3


...

[Fold 12] Epoch 4 train contrastive loss 0.0545 | time 102.71s
[Fold 12] Epoch 4 quick VAL (k=5) acc 91.73%
[Fold 12] New BEST saved -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\model_fold12.pth


[Fold 12] Epoch 5 train contrastive loss 0.0483 | time 102.40s
[Fold 12] Epoch 5 quick VAL (k=5) acc 93.07%
[Fold 12] New BEST saved -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\model_fold12.pth


[Fold 12] Epoch 6 train contrastive loss 0.0437 | time 102.13s
[Fold 12] Epoch 6 quick VAL (k=5) acc 88.53%
[Fold 12] EarlyStopping counter 1/3


[Fold 12] Epoch 7 train contrastive loss 0.0399 | time 106.01s
[Fold 12] Epoch 7 quick VAL (k=5) acc 91.60%
[Fold 12] EarlyStopping counter 2/3


[Fold 12] Epoch 8 train contrastive loss 0.0385 | time 103.46s
[Fold 12] Epoch 8 quick VAL (k=5) acc 90.27%
[Fold 12] EarlyStopping counter 3/3
[Fold 12] Early stopping.
[Fold 12] FINAL 5-way 5-shot VAL: 92.80% ± 0.65%
Completed 12-fold Siamese K=5. Metrics -> 12fold_ensemble_ckpt\12fold_optuna_siamese_5shot\metrics.json


In [ ]:
# 3.4.1.C (Siamese): Evaluate 12fold_optuna_siamese_5shot with MODEL VOTING 
import os, json, math, torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import torch.nn as nn
import torch.nn.functional as F

assert 'build_model' in globals(), "build_model(...) must be defined above."
assert 'train_loader' in globals(), "train_loader must exist (used to build prototypes)."
assert 'test_loader'  in globals(), "test_loader must exist."
assert 'device'       in globals(), "device must be set."

# --- Paths / config ---
DIR_SIAMESE = "12fold_ensemble_ckpt/12fold_optuna_siamese_5shot"
OUT_JSON    = os.path.join("12fold_ensemble_ckpt", "12fold_eval_results.json")
NUM_CLASSES = 102
EMB_DIM     = 512

# If you warm-started the backbone per fold from supervised checkpoints, point this there.
SUPERVISED_BACKBONE_DIR = None  # e.g., "12fold_ensemble_ckpt/k12_optuna_HP_largetrain"

# --- Projection head (must match training) ---
class ProjectionHead(nn.Module):
    def __init__(self, in_dim=2048, out_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, out_dim)
        )
    def forward(self, x):
        z = self.net(x)
        return F.normalize(z, dim=1)

def list_fold_ckpts(model_dir):
    ckpts = sorted([os.path.join(model_dir, f) for f in os.listdir(model_dir)
                    if f.endswith(".pth") and "model_fold" in f])
    if not ckpts:
        raise FileNotFoundError(f"No fold .pth files in {model_dir}")
    return ckpts

def get_backbone_for_fold(fold_idx):
    """
    Rebuild a backbone identical to Siamese training.
    - If SUPERVISED_BACKBONE_DIR is provided and contains model_fold{fold}.pth,
      we load that state_dict into the full model (strict=False) and take m[0] as backbone.
    - Else we use build_model(..., pretrained=True) and take m[0].
    """
    use_supervised = SUPERVISED_BACKBONE_DIR is not None
    m = build_model(num_classes=NUM_CLASSES, pretrained=not use_supervised)
    if use_supervised:
        sup_path = os.path.join(SUPERVISED_BACKBONE_DIR, f"model_fold{fold_idx}.pth")
        if os.path.exists(sup_path):
            state = torch.load(sup_path, map_location=device)
            try:
                m.load_state_dict(state, strict=False)
            except Exception:
                try:
                    m[0].load_state_dict(state, strict=False)
                except Exception as e:
                    raise RuntimeError(f"Failed loading supervised backbone for fold {fold_idx}: {e}")
        else:
            print(f"[Fold {fold_idx}] Warning: {sup_path} not found; falling back to ImageNet pretrained.")
    backbone = m[0].to(device).eval()
    for p in backbone.parameters():
        p.requires_grad = False
    return backbone

def load_proj_for_fold(ckpt_path):
    blob = torch.load(ckpt_path, map_location=device)
    if "proj" not in blob:
        raise KeyError(f"{ckpt_path} does not contain 'proj' weights")
    proj = ProjectionHead(2048, EMB_DIM).to(device).eval()
    proj.load_state_dict(blob["proj"])
    return proj

@torch.no_grad()
def build_class_prototypes(backbone, proj, data_loader):
    """
    Average normalized embeddings per class over the training split,
    then re-normalize the class means. Returns [C, D] on device.
    """
    sums   = [None] * NUM_CLASSES
    counts = [0] * NUM_CLASSES

    for imgs, labels in tqdm(data_loader, desc="Building prototypes", leave=False):
        imgs   = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            feats = backbone(imgs)   # [B, 2048]
            z     = proj(feats)      # [B, D], normalized
        for c in labels.unique().tolist():
            m = (labels == c)
            zc = z[m]
            if zc.numel() == 0:
                continue
            sums[c] = zc.sum(0) if sums[c] is None else (sums[c] + zc.sum(0))
            counts[c] += int(m.sum())

    protos = []
    for c in range(NUM_CLASSES):
        if counts[c] > 0:
            protos.append(F.normalize(sums[c] / counts[c], dim=0))
        else:
            protos.append(torch.zeros(EMB_DIM, device=device))
    return torch.stack(protos, dim=0)  # [C, D]

def _resolve_votes(votes_row: torch.Tensor,
                   avg_probs_row: torch.Tensor,
                   max_probs_row: torch.Tensor) -> int:
    """
    votes_row: [num_folds] int class indices
    avg_probs_row: [C] mean probability across folds for this sample
    max_probs_row: [C] max probability across folds for this sample
    Returns: chosen class index via:
      1) majority vote
      2) tie-breaker #1: highest mean prob among tied classes
      3) tie-breaker #2: highest single-model max prob among tied classes
      4) final: smallest class index (deterministic)
    """
    counts = torch.bincount(votes_row, minlength=NUM_CLASSES)
    max_count = counts.max()
    candidates = (counts == max_count).nonzero(as_tuple=False).squeeze(1)

    if candidates.numel() == 1:
        return int(candidates.item())

    # tie-breaker #1: highest mean probability
    cand_avg = avg_probs_row[candidates]
    top_avg = cand_avg.max()
    cand2 = candidates[(cand_avg == top_avg)]
    if cand2.numel() == 1:
        return int(cand2.item())

    # tie-breaker #2: highest single-model max probability
    cand_max = max_probs_row[cand2]
    top_max = cand_max.max()
    cand3 = cand2[(cand_max == top_max)]
    # final deterministic fallback
    return int(cand3.min().item())

@torch.no_grad()
def evaluate_siamese_dir(model_dir, label):
    fold_ckpts = list_fold_ckpts(model_dir)
    print(f"\n=== Evaluating {label} (Model Voting) ===")
    print(f"Found {len(fold_ckpts)} projection heads in: {model_dir}")

    # --- Build per-fold components ---
    per_fold = []
    for i, ckpt in enumerate(fold_ckpts, start=1):
        print(f"[{label}] Preparing fold {i}/{len(fold_ckpts)}")
        backbone = get_backbone_for_fold(i)
        proj     = load_proj_for_fold(ckpt)
        protos   = build_class_prototypes(backbone, proj, train_loader)
        per_fold.append((backbone, proj, protos))

    # --- Evaluate via per-model predictions + voting ---
    all_preds, all_labels = [], []
    for imgs, labels in tqdm(test_loader, desc=f"Testing ({label})"):
        imgs = imgs.to(device, non_blocking=True)

        fold_top1_list = []
        fold_probs_list = []

        for (backbone, proj, protos) in per_fold:
            with torch.cuda.amp.autocast(True):
                feats = backbone(imgs)          # [B, 2048]
                zq    = proj(feats)             # [B, D]
                sims  = zq @ protos.t()         # [B, C]
                probs = torch.softmax(sims, dim=1)  # [B, C]
            fold_probs_list.append(probs.detach().cpu())
            fold_top1_list.append(probs.argmax(dim=1).detach().cpu())

        # Stack across folds
        votes = torch.stack(fold_top1_list, dim=0).T.clone()        # [B, num_folds]
        probs_stack = torch.stack(fold_probs_list, dim=0)           # [num_folds, B, C]
        avg_probs   = probs_stack.mean(dim=0)                       # [B, C]
        max_probs   = probs_stack.max(dim=0).values                 # [B, C]

        # Resolve votes per sample with deterministic tie-breakers
        B = votes.size(0)
        for b in range(B):
            pred_b = _resolve_votes(
                votes_row=votes[b],
                avg_probs_row=avg_probs[b],
                max_probs_row=max_probs[b],
            )
            all_preds.append(pred_b)
        all_labels.extend(labels.numpy().tolist())

    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4, output_dict=True)
    macro_f1    = report["macro avg"]["f1-score"]
    weighted_f1 = report["weighted avg"]["f1-score"]

    print(f"[{label}] Test Acc: {acc:.4f} | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f}")

    # free GPU
    for backbone, proj, _ in per_fold:
        del backbone, proj
    del per_fold
    torch.cuda.empty_cache()

    return {
        "num_models": len(fold_ckpts),
        "test_accuracy": round(acc, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
        "models": fold_ckpts,
        "ensemble_method": "model_voting_with_prob_tiebreakers"
    }

# --- Run on 12fold_optuna_siamese_5shot and persist results ---
siamese_result = evaluate_siamese_dir(DIR_SIAMESE, "12fold_optuna_siamese_5shot")

results = {}
if os.path.exists(OUT_JSON):
    try:
        with open(OUT_JSON, "r") as f:
            results = json.load(f)
    except Exception:
        results = {}

results["12fold_optuna_siamese_5shot"] = siamese_result

os.makedirs(os.path.dirname(OUT_JSON), exist_ok=True)
with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"\nSaved siamese evaluation to {OUT_JSON}")



=== Evaluating 12fold_optuna_siamese_5shot (Model Voting) ===
Found 12 projection heads in: 12fold_ensemble_ckpt/12fold_optuna_siamese_5shot
[12fold_optuna_siamese_5shot] Preparing fold 1/12


Building prototypes:   0%|          | 0/32 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1177923387.py:91: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


[12fold_optuna_siamese_5shot] Preparing fold 2/12


[12fold_optuna_siamese_5shot] Preparing fold 3/12


[12fold_optuna_siamese_5shot] Preparing fold 4/12


...

[12fold_optuna_siamese_5shot] Preparing fold 10/12


[12fold_optuna_siamese_5shot] Preparing fold 11/12


[12fold_optuna_siamese_5shot] Preparing fold 12/12


Testing (12fold_optuna_siamese_5shot):   0%|          | 0/97 [00:00<?, ?it/s]C:\Users\Eugene Ian\AppData\Local\Temp\ipykernel_67224\1177923387.py:168: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
Testing (12fold_optuna_siamese_5shot): 100%|██████████| 97/97 [01:04<00:00,  1.50it/s]

[12fold_optuna_siamese_5shot] Test Acc: 0.4711 | Macro F1: 0.4739 | Weighted F1: 0.4768

Saved siamese evaluation to 12fold_ensemble_ckpt\12fold_eval_results.json
